In [1]:
# Source: https://huggingface.co/learn/nlp-course/en/chapter7/5?fw=pt
# https://github.com/rashi-bhansali/Abstractive-Text-Summarization-of-Hindi-Data-using-Transformers
# https://github.com/parvpareek/hindi-text-summarization

In [2]:
!pip install transformers==4.30
!pip install accelerate -U
!pip install rouge_score
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 981.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.30.0 which is incompatible.
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size

In [3]:
import pandas as pd
import torch
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import numpy as np
import nltk
import evaluate
rouge_score = evaluate.load("rouge")
nltk.download("punkt")
from transformers import DataCollatorForSeq2Seq
from nltk.tokenize import sent_tokenize

2024-04-25 04:03:33.249633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 04:03:33.249757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 04:03:33.514192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/dataset/hindi_train_final_TEXTRANK.csv")[["Article", "Summary"]].dropna())
val_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/dataset/hindi_val_final_TEXTRANK.csv")[["Article", "Summary"]].dropna())
test_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/dataset/hindi_test_final_TEXTRANK.csv")[["Article", "Summary"]].dropna())

In [6]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")

tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [8]:
def tokenize_helper(dat):
    model_inputs = tokenizer(
        dat["Article"],
        truncation = True,
        max_length=1024
    )
    true_sent = tokenizer(dat["Summary"], truncation = True, max_length=1024)
    model_inputs["labels"] = true_sent["input_ids"]
    return model_inputs

tokenized_train = train_data.map(tokenize_helper, batched=True)
tokenized_val = val_data.map(tokenize_helper, batched=True)

Map:   0%|          | 0/19087 [00:00<?, ? examples/s]

Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)
tokenized_train = tokenized_train.remove_columns(train_data.column_names)
tokenized_val = tokenized_val.remove_columns(val_data.column_names)

In [10]:
batch_size = 4
epochs = 10
args = Seq2SeqTrainingArguments(
    output_dir = "/kaggle/working/",
    evaluation_strategy = "epoch",
    learning_rate = 1e-3,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_total_limit=3,
    predict_with_generate=True,
    num_train_epochs = epochs,
    report_to="none"
)

In [11]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.706800,1.534676,13.297500,4.123400,12.899300,12.896100
2,1.447900,1.472715,14.480400,4.440500,14.089400,14.091300
3,1.215700,1.464413,15.219400,5.366900,14.724700,14.733600
4,1.063600,1.482312,15.959900,5.818500,15.436100,15.420700
5,0.905700,1.516759,16.458100,6.088400,15.875100,15.903700
6,0.766300,1.603036,15.737500,5.789500,15.170500,15.257400
7,0.643700,1.705722,15.449300,6.024500,14.807600,14.842100
8,0.549300,1.770645,16.516900,6.271400,15.926700,15.962900
9,0.458900,1.884269,16.260200,6.210100,15.671500,15.696400
10,0.402900,1.952927,16.168100,6.250300,15.501600,15.596200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=23860, training_loss=0.9231665752759424, metrics={'train_runtime': 15585.3638, 'train_samples_per_second': 12.247, 'train_steps_per_second': 1.531, 'total_flos': 7.002134140030157e+16, 'train_loss': 0.9231665752759424, 'epoch': 10.0})

In [13]:
trainer.evaluate()

{'eval_loss': 1.9529266357421875,
 'eval_rouge1': 16.1681,
 'eval_rouge2': 6.2503,
 'eval_rougeL': 15.5016,
 'eval_rougeLsum': 15.5962,
 'eval_runtime': 73.0827,
 'eval_samples_per_second': 14.518,
 'eval_steps_per_second': 1.82,
 'epoch': 10.0}